# **TRAI - DATA ANALYSIS WITH PYTHON - CASE STUDY**

---
Authors:
*   Furkan Memiş
*   Tunca Tezcan
*   Ömer Faruk Dilbaz




In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as bs
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# BIST100 SECTION

Notlar:
  Sektöre göre gruplama yapılabilir.
  2'li plotlama ile görselleştirme yapılabilir.

In [ ]:
from numpy import float32
url = 'https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/Temel-Degerler-Ve-Oranlar.aspx?endeks=01#page-1'
response = requests.get(url)
response.text
soup = bs(response.text, 'html.parser')
table = soup.find('table', {'id' : 'summaryBasicData'})
table = pd.read_html(str(table))
dtypes = {'Kod':'str',
        'Hisse Adı':'str',
        'Sektör':'str',
        'Kapanış (TL)':'float',
        'Piyasa Değeri (mn TL)':'float',
        'Piyasa Değeri (mn $)':'float',
        'Halka Açıklık Oranı (%)':'float',
        'Sermaye (mn TL)':'float'
        }
df = pd.DataFrame(table[0])
convert_names = ['Piyasa Değeri (mn TL)', 'Piyasa Değeri (mn $)','Sermaye (mn TL)','Kapanış (TL)']
df[convert_names] = df[convert_names].applymap(lambda x: x.replace('.', '').replace(',', '.'))
df = df.astype(dtypes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Kod                      100 non-null    object 
 1   Hisse Adı                100 non-null    object 
 2   Sektör                   100 non-null    object 
 3   Kapanış (TL)             100 non-null    float64
 4   Piyasa Değeri (mn TL)    100 non-null    float64
 5   Piyasa Değeri (mn $)     100 non-null    float64
 6   Halka Açıklık Oranı (%)  100 non-null    float64
 7   Sermaye (mn TL)          100 non-null    float64
dtypes: float64(5), object(3)
memory usage: 6.4+ KB


In [ ]:
value_s = dict(df['Sektör'].value_counts())
value_s

{'Elektrik': 13,
 'Holdingler': 10,
 'Bankacılık': 9,
 'Otomotiv': 8,
 'Demir-Çelik Temel': 6,
 'GYO': 6,
 'Gıda': 5,
 'Perakande - Ticaret': 4,
 'Çimento': 4,
 'Sağlık ve İlaç': 3,
 'Havayolları ve Hizm.': 3,
 'Tarım Kimyasalları': 3,
 'Dayanıklı Tüketim': 3,
 'Endüstriyel Tekstil': 3,
 'İletişim': 2,
 'Petrol': 2,
 'Madencilik': 2,
 'Meşrubat / İçecek': 2,
 'Demir-Çelik Döküm': 2,
 'İnşaat- Taahhüt': 2,
 'Kimyasal Ürün': 2,
 'Spor': 1,
 'Tekstil Entegre': 1,
 'Otomotiv Parçası': 1,
 'Savunma': 1,
 'Cam': 1,
 'Sigorta': 1}

In [ ]:
type(value_s.keys())

dict_keys

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=df['Sektör'], values=df['Piyasa Değeri (mn TL)'],title='Sektör/Piyasa Değeri TL'),1, 1)
fig.add_trace(go.Pie(labels=list(value_s.keys()), values=list(value_s.values()),title='Sektör/Şirket Sayısı'),1, 2)
fig.update_layout(font=dict(size=14))
fig.show()

***Sektör/Piyasa Değeri TL grafiğine baktığımızda en yüksek hacme sahip sektörün Bankacılık olduğunu görmekteyiz. Sektör/Şirket Sayısı grafiğine baktığımızda da BIST100'de en fazla şirkete sahip olan sektörün Holdingler olduğunu göremekteyiz. BIST100 şirketlerine baktığımızda, sektörel bazda dağılımlarını incelediğimizde; en çok şirkete sahip olan sektörün piyasa değeri açısından lider konumda olmadığını iş dünyası perspektifinden gözlemlemekteyiz.***

In [ ]:
new_df = df.groupby('Sektör').agg({'Halka Açıklık Oranı (%)': 'mean', 'Piyasa Değeri (mn TL)': 'sum','Sermaye (mn TL)':'sum'})
new_df['Halka Açıklık Oranı (%)'] = new_df['Halka Açıklık Oranı (%)']/10
new_df['Piyasa Değeri (bn TL)'] = new_df['Piyasa Değeri (mn TL)']/1000
new_df['Sermaye (bn TL)'] = new_df['Sermaye (mn TL)']/1000
new_df['Piyasa Değeri (% TL)'] = (new_df['Piyasa Değeri (mn TL)']/new_df['Piyasa Değeri (mn TL)'].sum())*100
new_df


,Halka Açıklık Oranı (%),Piyasa Değeri (mn TL),Sermaye (mn TL),Piyasa Değeri (bn TL),Sermaye (bn TL),Piyasa Değeri (% TL)
Sektör,,,,,,
Bankacılık,29.944444,614825.9,52107.8,614.8259,52.1078,16.481976
Cam,48.900000,112113.6,3063.2,112.1136,3.0632,3.005491
Dayanıklı Tüketim,24.600000,89539.2,11712.0,89.5392,11.7120,2.400327
Demir-Çelik Döküm,44.100000,8282.8,7800.0,8.2828,7.8000,0.222042
Demir-Çelik Temel,39.983333,217388.1,23288.0,217.3881,23.2880,5.827642
Elektrik,30.338462,193391.0,43423.4,193.3910,43.4234,5.184339
Endüstriyel Tekstil,28.933333,247396.6,7485.6,247.3966,7.4856,6.632097
GYO,42.016667,47622.6,20251.9,47.6226,20.2519,1.276645
Gıda,30.100000,35256.1,21584.0,35.2561,21.5840,0.945130


In [ ]:
new_df['Piyasa Değeri (mn TL)'].sum()

3730292.4999999995

In [ ]:

fig = go.Figure()
fig.add_trace(go.Bar(y=new_df.index,x=new_df['Halka Açıklık Oranı (%)'],name='Halka Açıklık Oranı', orientation='h'))
fig.add_trace(go.Bar(y=new_df.index,x=new_df['Piyasa Değeri (% TL)'],name='Piyasa Değeri', orientation='h'))
fig.update_layout(title = 'Halka Açıklık Oranı ve Piyasa Değeri Yüzdelik Karşılaştırması',
                  xaxis_title = 'Sektör',
                  yaxis_title = "Yüzedelik Değerler",
                  width=700,height=1000
                  )
fig.show()

***BIST100 şirketlerini, sektör bazlı incelediğimizde halka açıklık oranı olarak en yüksek paya Tekstil Entegre sahipken; piyasa değeri açısından incelediğimizde en yüksek paya sahip sektör Bankacılık sektörüdür. Sonuç olarak sektörlerin halka açıklık oran yüzdeleri ile piyasa değer yüzdeleri arasında doğrudan bir ilişki yoktur.***

# BIST5 and BIST100 COMPANY ANALYSIS SECTION
Companys: EGEEN, KARSN, TMSN, TOASO, TTRAK

In [ ]:
liste = ['EGEEN.IS', 'KARSN.IS', 'TMSN.IS', 'TOASO.IS', 'TTRAK.IS', 'XU100.IS']
data = yf.download(liste, start='2021-01-04', end='2021-12-31')
data

[*********************100%***********************]  6 of 6 completed


Adj Close                                             \
               EGEEN.IS KARSN.IS    TMSN.IS   TOASO.IS    TTRAK.IS   
Date                                                                 
2021-01-04  1536.786743     3.57  14.190000  28.038450  146.718643   
2021-01-05  1526.995972     3.60  13.870000  28.265610  148.559647   
2021-01-06  1518.013672     3.50  13.910000  28.379192  154.482803   
2021-01-07  1517.115479     3.60  14.050000  28.444096  152.962006   
2021-01-08  1612.148560     3.59  14.560000  29.076908  147.439026   
...                 ...      ...        ...        ...         ...   
2021-12-24  1649.500000     3.47  17.580000  63.698742  178.538177   
2021-12-27  1628.997803     3.53  19.330000  65.968918  179.396545   
2021-12-28  1623.219849     3.41  19.670000  63.832272  179.396545   
2021-12-29  1639.249023     3.45  20.059999  66.236000  183.259155   
2021-12-30  1618.746460     3.39  18.930000  68.995834  183.688324   

                               Close                                 ...  \
               XU100.IS     EGEEN.IS KARSN.IS    TMSN.IS   TOASO.IS  ...   
Date                                                                 ...   
2021-01-04  1495.400024  1710.900024     3.57  14.190000  34.560001  ...   
2021-01-05  1489.599976  1700.000000     3.60  13.870000  34.840000  ...   
2021-01-06  1505.400024  1690.000000     3.50  13.910000  34.980000  ...   
2021-01-07  1522.199951  1689.000000     3.60  14.050000  35.060001  ...   
2021-01-08  1540.599976  1794.800049     3.59  14.560000  35.840000  ...   
...                 ...          ...      ...        ...        ...  ...   
2021-12-24  1891.300049  1770.000000     3.47  17.580000  71.550003  ...   
2021-12-27  1897.099976  1748.000000     3.53  19.330000  74.099998  ...   
2021-12-28  1850.500000  1741.800049     3.41  19.670000  71.699997  ...   
2021-12-29  1895.500000  1759.000000     3.45  20.059999  74.400002  ...   
2021-12-30  1863.699951  1737.000000     3.39  18.930000  77.500000  ...   

                 Open                                       Volume             \
              TMSN.IS   TOASO.IS    TTRAK.IS     XU100.IS EGEEN.IS   KARSN.IS   
Date                                                                            
2021-01-04  14.100000  34.000000  188.399994  1482.199951    46530  203980281   
2021-01-05  14.160000  34.680000  183.199997  1492.199951    79252  296933161   
2021-01-06  14.020000  35.060001  190.399994  1501.599976    48966  170722812   
2021-01-07  14.020000  35.040001  194.000000  1511.599976    46078  119093258   
2021-01-08  14.120000  35.259998  191.100006  1526.500000   294470  104442924   
...               ...        ...         ...          ...      ...        ...   
2021-12-24  16.940001  66.800003  195.000000  1820.699951    40957   32153629   
2021-12-27  17.799999  73.949997  208.000000  1898.699951    47581   42494956   
2021-12-28  19.469999  75.000000  209.500000  1913.300049   122045   21695297   
2021-12-29  19.799999  71.500000  206.899994  1847.300049    42240   24717699   
2021-12-30  20.200001  75.500000  214.000000  1916.300049    28104   17715130   

                                                      
             TMSN.IS TOASO.IS TTRAK.IS      XU100.IS  
Date                                                  
2021-01-04   6110639  2775413   838204  5.172334e+09  
2021-01-05   4071087  4066306   620734  6.659180e+09  
2021-01-06   2822124  2411356   646448  5.330873e+09  
2021-01-07   4418340  2184018   601630  6.175892e+09  
2021-01-08  15798308  2278684   557685  5.500828e+09  
...              ...      ...      ...           ...  
2021-12-24   2140708  3944590   416624  2.530669e+09  
2021-12-27   5015736  7411759   758007  3.014803e+09  
2021-12-28   6067847  3166074   329863  2.584687e+09  
2021-12-29   5203082  3348372   643580  2.570994e+09  
2021-12-30   3267099  5212054   511646  2.341048e+09  

[248 rows x 36 columns]

In [ ]:
kapanis_fiyatlari = data['Adj Close']
kapanis_fiyatlari

,EGEEN.IS,KARSN.IS,TMSN.IS,TOASO.IS,TTRAK.IS,XU100.IS
Date,,,,,,
2021-01-04,1536.786743,3.57,14.190000,28.038450,146.718643,1495.400024
2021-01-05,1526.995972,3.60,13.870000,28.265610,148.559647,1489.599976
2021-01-06,1518.013672,3.50,13.910000,28.379192,154.482803,1505.400024
2021-01-07,1517.115479,3.60,14.050000,28.444096,152.962006,1522.199951
2021-01-08,1612.148560,3.59,14.560000,29.076908,147.439026,1540.599976
...,...,...,...,...,...,...
2021-12-24,1649.500000,3.47,17.580000,63.698742,178.538177,1891.300049
2021-12-27,1628.997803,3.53,19.330000,65.968918,179.396545,1897.099976
2021-12-28,1623.219849,3.41,19.670000,63.832272,179.396545,1850.500000


In [ ]:
for comp in kapanis_fiyatlari.columns:
  kapanis_fiyatlari[comp] = kapanis_fiyatlari[comp]/kapanis_fiyatlari[comp].iloc[0]

<ipython-input-129-426274eb5c52>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
px.line(kapanis_fiyatlari,y=liste,title="5 Şirketin Normalleştirilmiş Hisse Senedi Fiyatları ile BIST100 Karşılaştırması")

***Son çeyreğe kadar 5 şirket ve BIST100 değerlerinin dalgalı bir seyir izlediğini görmekteyiz. Bununla birlikte, son çeyrekte TOASO (Tofaş Türk Otomobil Fabrikası) diğer şirketlerden ayrışarak artış eğilimi sergileyip, yaklaşık olarak iki kat değere ulaştığı gözlemlenmektedir. Diğer 4 şirket ve BIST100 endeksi ise dalgalı seyrine devam etmektedir.***

# BIST5 vs BIST100 SECTION


In [ ]:
liste = ['EGEEN.IS', 'KARSN.IS', 'TMSN.IS', 'TOASO.IS', 'TTRAK.IS', 'XU100.IS']

baslangic_zamani = '2021-01-01'
bitis_zamani = '2021-12-31'

BIST = yf.download(liste, start=baslangic_zamani, end=bitis_zamani)

BIST

[*********************100%***********************]  6 of 6 completed


Adj Close                                             \
               EGEEN.IS KARSN.IS    TMSN.IS   TOASO.IS    TTRAK.IS   
Date                                                                 
2021-01-04  1536.786743     3.57  14.190000  28.038446  146.718643   
2021-01-05  1526.995972     3.60  13.870000  28.265610  148.559631   
2021-01-06  1518.013672     3.50  13.910000  28.379190  154.482819   
2021-01-07  1517.115356     3.60  14.050000  28.444098  152.962006   
2021-01-08  1612.148682     3.59  14.560000  29.076906  147.439026   
...                 ...      ...        ...        ...         ...   
2021-12-24  1649.499878     3.47  17.580000  63.698738  178.538177   
2021-12-27  1628.997803     3.53  19.330000  65.968918  179.396545   
2021-12-28  1623.219849     3.41  19.670000  63.832272  179.396545   
2021-12-29  1639.248901     3.45  20.059999  66.236000  183.259155   
2021-12-30  1618.746582     3.39  18.930000  68.995834  183.688324   

                               Close                                 ...  \
               XU100.IS     EGEEN.IS KARSN.IS    TMSN.IS   TOASO.IS  ...   
Date                                                                 ...   
2021-01-04  1495.400024  1710.900024     3.57  14.190000  34.560001  ...   
2021-01-05  1489.599976  1700.000000     3.60  13.870000  34.840000  ...   
2021-01-06  1505.400024  1690.000000     3.50  13.910000  34.980000  ...   
2021-01-07  1522.199951  1689.000000     3.60  14.050000  35.060001  ...   
2021-01-08  1540.599976  1794.800049     3.59  14.560000  35.840000  ...   
...                 ...          ...      ...        ...        ...  ...   
2021-12-24  1891.300049  1770.000000     3.47  17.580000  71.550003  ...   
2021-12-27  1897.099976  1748.000000     3.53  19.330000  74.099998  ...   
2021-12-28  1850.500000  1741.800049     3.41  19.670000  71.699997  ...   
2021-12-29  1895.500000  1759.000000     3.45  20.059999  74.400002  ...   
2021-12-30  1863.699951  1737.000000     3.39  18.930000  77.500000  ...   

                 Open                                       Volume             \
              TMSN.IS   TOASO.IS    TTRAK.IS     XU100.IS EGEEN.IS   KARSN.IS   
Date                                                                            
2021-01-04  14.100000  34.000000  188.399994  1482.199951    46530  203980281   
2021-01-05  14.160000  34.680000  183.199997  1492.199951    79252  296933161   
2021-01-06  14.020000  35.060001  190.399994  1501.599976    48966  170722812   
2021-01-07  14.020000  35.040001  194.000000  1511.599976    46078  119093258   
2021-01-08  14.120000  35.259998  191.100006  1526.500000   294470  104442924   
...               ...        ...         ...          ...      ...        ...   
2021-12-24  16.940001  66.800003  195.000000  1820.699951    40957   32153629   
2021-12-27  17.799999  73.949997  208.000000  1898.699951    47581   42494956   
2021-12-28  19.469999  75.000000  209.500000  1913.300049   122045   21695297   
2021-12-29  19.799999  71.500000  206.899994  1847.300049    42240   24717699   
2021-12-30  20.200001  75.500000  214.000000  1916.300049    28104   17715130   

                                                      
             TMSN.IS TOASO.IS TTRAK.IS      XU100.IS  
Date                                                  
2021-01-04   6110639  2775413   838204  5.172334e+09  
2021-01-05   4071087  4066306   620734  6.659180e+09  
2021-01-06   2822124  2411356   646448  5.330873e+09  
2021-01-07   4418340  2184018   601630  6.175892e+09  
2021-01-08  15798308  2278684   557685  5.500828e+09  
...              ...      ...      ...           ...  
2021-12-24   2140708  3944590   416624  2.530669e+09  
2021-12-27   5015736  7411759   758007  3.014803e+09  
2021-12-28   6067847  3166074   329863  2.584687e+09  
2021-12-29   5203082  3348372   643580  2.570994e+09  
2021-12-30   3267099  5212054   511646  2.341048e+09  

[248 rows x 36 columns]

In [ ]:
BIST = BIST['Adj Close']

In [ ]:
BIST['XU100.IS'].fillna(value=BIST['XU100.IS'].mean(), inplace=True)

<ipython-input-133-0aadb4e26bb8>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
BIST.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 248 entries, 2021-01-04 to 2021-12-30
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   EGEEN.IS  248 non-null    float64
 1   KARSN.IS  248 non-null    float64
 2   TMSN.IS   248 non-null    float64
 3   TOASO.IS  248 non-null    float64
 4   TTRAK.IS  248 non-null    float64
 5   XU100.IS  248 non-null    float64
dtypes: float64(6)
memory usage: 13.6 KB


In [ ]:
for column in BIST.columns:
  BIST[column+" Normalize"] = BIST[column]/BIST[column].iloc[0]

BIST['BIST5'] = (BIST.iloc[:, 6:11].sum(axis=1)) / 5
BIST

<ipython-input-135-863b6068e61b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-135-863b6068e61b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-135-863b6068e61b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-135-863

,EGEEN.IS,KARSN.IS,TMSN.IS,TOASO.IS,TTRAK.IS,XU100.IS,EGEEN.IS Normalize,KARSN.IS Normalize,TMSN.IS Normalize,TOASO.IS Normalize,TTRAK.IS Normalize,XU100.IS Normalize,BIST5
Date,,,,,,,,,,,,,
2021-01-04,1536.786743,3.57,14.190000,28.038446,146.718643,1495.400024,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2021-01-05,1526.995972,3.60,13.870000,28.265610,148.559631,1489.599976,0.993629,1.008403,0.977449,1.008102,1.012548,0.996121,1.000026
2021-01-06,1518.013672,3.50,13.910000,28.379190,154.482819,1505.400024,0.987784,0.980392,0.980268,1.012153,1.052919,1.006687,1.002703
2021-01-07,1517.115356,3.60,14.050000,28.444098,152.962006,1522.199951,0.987200,1.008403,0.990134,1.014468,1.042553,1.017922,1.008552
2021-01-08,1612.148682,3.59,14.560000,29.076906,147.439026,1540.599976,1.049039,1.005602,1.026075,1.037037,1.004910,1.030226,1.024533
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,1649.499878,3.47,17.580000,63.698738,178.538177,1891.300049,1.073343,0.971989,1.238901,2.271836,1.216875,1.264745,1.354589
2021-12-27,1628.997803,3.53,19.330000,65.968918,179.396545,1897.099976,1.060003,0.988796,1.362227,2.352802,1.222725,1.268624,1.397310
2021-12-28,1623.219849,3.41,19.670000,63.832272,179.396545,1850.500000,1.056243,0.955182,1.386188,2.276598,1.222725,1.237462,1.379387


In [ ]:
BIST = BIST[['XU100.IS Normalize',	'BIST5']]
BIST.rename(columns={"XU100.IS Normalize": "BIST100 Normalize", "BIST5": "BIST5 Normalize"},inplace=True)

<ipython-input-136-0c633e0e2056>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
dolar_data = yf.download('USDTRY=X', start='2021-01-04', end='2021-12-31')
dolar_data = pd.DataFrame(dolar_data['Adj Close'])
dolar_data['USDTRY=X'] = dolar_data/dolar_data.iloc[0]
dolar_data['USDTRY=X']

[*********************100%***********************]  1 of 1 completed


Date
2021-01-04    1.000000
2021-01-05    0.998396
2021-01-06    0.993323
2021-01-07    0.982014
2021-01-08    0.985630
                ...   
2021-12-24    1.535404
2021-12-27    1.453849
2021-12-28    1.549899
2021-12-29    1.587845
2021-12-30    1.700469
Name: USDTRY=X, Length: 259, dtype: float64

In [ ]:
BIST

,BIST100 Normalize,BIST5 Normalize
Date,,
2021-01-04,1.000000,1.000000
2021-01-05,0.996121,1.000026
2021-01-06,1.006687,1.002703
2021-01-07,1.017922,1.008552
2021-01-08,1.030226,1.024533
...,...,...
2021-12-24,1.264745,1.354589
2021-12-27,1.268624,1.397310
2021-12-28,1.237462,1.379387


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=BIST.index,y=BIST['BIST100 Normalize'],name='BIST100'))
fig.add_trace(go.Scatter(x=BIST.index,y=BIST['BIST5 Normalize'],name='BIST5'))
fig.add_trace(go.Scatter(x=dolar_data.index,y=dolar_data['USDTRY=X'],name='USD/TRY'))
fig.update_layout(title="USD/TRY değişimiyle BIST5 ve BIST100  Endekslerinin Karşılaştırması",xaxis_title = 'Tarih', xaxis_dtick="M1",yaxis_title = "Normalize Değerler")
fig.show()

***Ekim ayına kadar BIST5 ve USD/TRY değerleri dalgalı bir seyir izlerken, BIST100 düşüş eğilimi göstermiştir. Ekim ayından Aralık ortasına kadar hızlanarak yükselen değerler, sonrasında sert bir düşüş yaşayarak dönemi kapatmıştır. USD/TRY'nin artışı, BIST5 ve BIST100 endekslerine olumlu yönde etki ettiği gözlemlenmektedir. Genel olarak, BIST100 ve BIST5 şirketleri dolar üzerinden gelir elde eden şirketler olarak bilinir.***

In [ ]:
BIST['ay_yıl'] = BIST.index.strftime('%Y-%m')
BIST = BIST.pivot_table(values=['BIST100 Normalize','BIST5 Normalize'], index='ay_yıl',aggfunc='last')


<ipython-input-140-6faebc16fc28>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
BIST['BIST5 Degisim'] = BIST['BIST5 Normalize'].diff(1)
BIST['BIST5 Degisim %'] = (BIST['BIST5 Degisim']/BIST['BIST5 Normalize'].shift(1))*100

In [ ]:
BIST['BIST100 Degisim'] = BIST['BIST100 Normalize'].diff(1)
BIST['BIST100 Degisim %'] = (BIST['BIST100 Degisim']/BIST['BIST100 Normalize'].shift(1))*100

In [ ]:
trace1 = go.Bar(
   x = BIST.index,
   y = BIST['BIST100 Degisim %'],
   name = 'BIST 100',
   text=round(BIST['BIST100 Degisim %'],2),
   textangle=0
)
trace2 = go.Bar(
   x = BIST.index,
   y = BIST['BIST5 Degisim %'],
   name = 'BIST 5',
   text=round(BIST['BIST5 Degisim %'],2),
   textangle=0
)
data = [trace1, trace2]
layout = go.Layout(barmode = 'group',title="BIST100 ve BIST5 Endekslerinin Aylık %'lik Değişim Grafiği",xaxis_title="Tarih",yaxis_title="% Değişim")
fig = go.Figure(data = data, layout = layout)
fig.show()


## ***BIST100 endeksi ve BIST5 endeksindeki değişimin birbirlerine önemli ölçüde etki etmediği gözlemlenmiştir.***